In [4]:
# Dependencies
import os
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
import plotly.express as px  # For interactive map
import matplotlib.patheffects as path_effects  # Import for text effects
from IPython.display import display  # For nice printing of dataframes
from matplotlib.lines import Line2D
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#Read in the data
data_frames = []

# Download latest version of the dataset
dataset_name = "osmi/mental-health-in-tech-survey"
if not os.path.exists(f"{dataset_name}.zip"): # Check if zip already exists
    os.system(f"kaggle datasets download -d {dataset_name}")


# Extract the zipfile
csv_filename = "survey.csv"  # The CSV inside the zip
if not os.path.exists(csv_filename): 
    with zipfile.ZipFile(f"{dataset_name}.zip", 'r') as zip_ref:
        zip_ref.extractall(".")

data_paths = [csv_filename]

# Rename the columns for clarity
col_renames = {
       'Age':'Age', 'Gender':'Gender', 'Country':'Country', 'state':'state', 'self_employed':'self-employed',
       'family_history':'family history', 'treatment':'treatment', 'work_interfere':'work interfere', 'no_employees':'number of employees',
       'remote_work':'remote work', 'tech_company':'tech company', 'benefits':'benefits', 'care_options':'care options',
       'wellness_program':'wellness program', 'seek_help':'seek help', 'anonymity':'anonymity', 'leave':'leave',
       'mental_health_consequence':'mental health consequence', 'phys_health_consequence':'physical health consequence', 'coworkers':'coworkers',
       'supervisor':'supervisor', 'mental_health_interview':'mental health interview', 'phys_health_interview':'physical health interview',
       'mental_vs_physical':'mental vs physical', 'obs_consequence':'obs consequence',
}
col_to_keep = col_renames.keys()

# Examining the characteristics of the dataset and performing initial cleaning of the dataset
for path in data_paths:
  print("\nReading data file: ", path, "\n")
  df = pd.read_csv(path)
  print('Shape - default: ', df.shape, '\n')

  # There are a lot of columns, many with a lot of missing values. Let's filter those columns and keep only those columns which have less than 90% 
  # missing values.
  max_na_filter = (0.9 * len(df))
  df = df.loc[:,(df.isnull().sum(axis = 0) <= max_na_filter)]
  print('Shape - after removing columns with lots of missing values: ', df.shape, '\n')

  # Removing the unneeded columns
  col_to_drop = [item for item in df.columns if item not in col_to_keep]
  df.drop(columns = col_to_drop, inplace = True, errors = 'ignore')
  print('Shape - after removing unneeded columns: ', df.shape, '\n')

  # Renaming columns
  df.rename(columns = col_renames, inplace = True) 

  # Storing dataframes in list
  data_frames.append(df)

# Examining characteristics of the dataframe
data_df = pd.concat(data_frames, ignore_index = True)
print('Shape - appended data: ', data_df.shape, '\n')
print('Shape - concatenated data: ', df.shape, '\n')
print('Sample of the filtered and combined dataframe')
# print(data_df.head(10), '\n')
display(data_df.head(10))
print('\n')

# Check the number of empty rows in the dataframe
print('Check the number of empty rows in the dataframe')
print(data_df.isna().sum().sort_values(), '\n')

#### Data Cleaning and Preprocessing

# Focus on tech-related employees 
# Since we are targetting tech related employees, setting default input values will make incorrect analysis, 
# hence removing the empty value rows 
# data_df = data_df[data_df['tech company'].notna()]
data_df.dropna(subset=['tech company'], inplace=True)
# Examine the values in the 'tech company' column
# print(data_df['tech company'].value_counts())
print('Values in the tech company column:  \n', data_df.value_counts(subset=['tech company']), '\n')
# Remove the rows belonging to survey respondents who do not have a tech related job
data_df.drop(data_df[data_df['tech company'] == 0.0].index, inplace = True)
# Now, since all of our data contains records of respondents who have tech related jobs, we can safely remove the tech company column
data_df.drop(columns = 'tech company', inplace = True)
df = data_df
#print(df.head(), '\n')
print('Sample of data frame after retaining only survey respondents who worked for a tech company')
display(data_df.head())
print('\n')

# Handle missing values strategically
#df['self-employed'].fillna('Other', inplace=True)
df.fillna({'self-employed':'Other'}, inplace=True)
#df['state'].fillna('Other', inplace=True)
df.fillna({'state':'Other'}, inplace=True)
#df['work interfere'].fillna('Other', inplace=True)
df.fillna({'work interfere':'Other'}, inplace=True)

# Handle Age outliers
# Examine the 'Age' values within the dataframe
print('Examine the Age values in the dataframe: \n', data_df['Age'].describe().astype(int), '\n')
# Address irregularites in 'Age' values
df['Age'] = df['Age'].astype(float)
mean_age = df[(df['Age'] >= 18) & (df['Age'] <= 75)]['Age'].mean()
df.loc[(df['Age'] < 18) | (df['Age'] > 75), 'Age'] = mean_age
print('Examine age values in the dataframe after age irregularities have been addressed: \n',  data_df['Age'].describe().astype(int), '\n')


#### Standardize Categorical Features

# Standardize Gender
# Map gender values to me either male, female, or other.
gender_male = ['male', 'M', 'Make', 'Male ', 'Man', 'Cis Male', 'msle', 'male leaning androgynous', 'Mail', 'Malr', 'Cis Man', 'Guy (-ish) ^_^', 'ostensibly male, unsure what that really means', 'Male (CIS)', 'Mal', 'maile', 'Male-ish', 'Androgyne', 'something kinda male?', 'm', 'man', 'male/he/him', 'let\'s keep it simple and say \"male\"', 'mostly male', 'masculine', 'identify as male', 'masculino', 'cishet male', 'cis male', 'mail', 'male-ish', 'cis-male', 'male (cis)', 'cis hetero male', 'dude', 'cisgender male', 'male, born with xy chromosoms', 'swm', 'ostensibly male']
gender_female = ['female', 'f', 'F', 'woman', 'Woman', 'Female ', 'Female (cis)', 'cis-female/femme', 'Femake', 'Cis Female', 'female, she/her', 'femile', 'female (cis)', 'f, cisgender', 'cisgendered woman', 'femmina', 'cis female', 'cis woman', 'cis-female', 'genderqueer demigirl', 'female (cisgender)', 'my sex is female.', 'femail', 'femalw', 'nonbinary/femme', 'cisgender female', 'she/her/they/them', '*shrug emoji* (f)',  'female/gender non-binary.', 'i identify as female']
gender_other = ['agender', 'Female (trans)', 'Genderqueer', 'A little about you', 'queer', 'Neuter', 'Trans woman', 'Agender', 'fluid', 'Enby', 'All', 'Nah', 'queer/she/they', 'Trans-female', 'p', 'nonbinary', 'nb', 'b', 'gender non-conforming woman', 'demiguy', 'trans non-binary/genderfluid', 'other', 'afab non-binary', 'sometimes', 'questioning', 'none', 'trans man', 'trans woman', 'trans female', 'non-binary/agender', 'make', 'agender trans woman', 'transfeminine', 'genderqueer/non-binary', 'non binary', 'contextual', 'agender/genderfluid', 'non-binary', 'genderfluid', 'god king of the valajar', 'uhhhhhhhhh fem genderqueer?', 'transgender', 'genderqueer', 'homem cis']
# lets transfor gender in simpler form
data_df['Gender'] = data_df['Gender'].replace(gender_male, 'Male')
data_df['Gender'] = data_df['Gender'].replace(gender_female, 'Female')
data_df['Gender'] = data_df['Gender'].replace(gender_other, 'Other')
# Contents of the 'Gender' column after reducing gender categories to male, female, or other
print('Contents of the Gender column after reducing gender categories:  \n', data_df['Gender'].value_counts(), '\n')

# Standardize Benefits (Available Mental Health Benefits)
# Check the number of empty rows in dataframe again
print('Check the number of empty rows in the dataframe again:  \n', data_df.isna().sum().sort_values(), '\n')
# Examine the values in the 'benefits' column
print('Examine the values in the benefits column:  \n', data_df['benefits'].value_counts(), '\n')
# Update Not eligible for coverage and NA to be the same as No in the 'Benefits' column
data_df.loc[data_df['benefits'] == 'Not eligible for coverage / NA' , 'benefits'] = 'No'
print('Contents of the benefits columns after making Not eligible for coverage and NA to be the same as No:  \n', data_df['benefits'].value_counts(), '\n')


#### Feature Reduction

# Examine the unique data types in the dataframe
print('Unique data types in the dataframe:  \n', data_df.nunique(), '\n')   
# Examine additional information on the dataframe
print('Additional information on the dataframe:  \n', data_df.info(), '\n')
# Now all missing data is fixed, examine the records in the dataframe
print('Records in the dataframe after accommodating missing data:  \n', data_df.shape, '\n')

# Removal of possible duplicate rows
data_df.drop_duplicates(inplace=True)
data_df.head()

# Get value counts for each categorical column
for col in data_df.select_dtypes(include='object'):
    print(data_df[col].value_counts(), '\n')

# Save cleaned data for later processing
try:
    data_df.to_csv('cleaned_data_df.csv', index=False)  # Save as CSV
    print("cleaned_data_df saved successfully!")
except Exception as e:
    print(f"Error saving cleaned_data_df: {e}")

try:
    df.to_csv('cleaned_df.csv', index=False)  # Save as CSV
    print("cleaned_df saved successfully!")
except Exception as e:
    print(f"Error saving cleaned_df: {e}")



Reading data file:  survey.csv 

Shape - default:  (1259, 27) 

Shape - after removing columns with lots of missing values:  (1259, 27) 

Shape - after removing unneeded columns:  (1259, 25) 

Shape - appended data:  (1259, 25) 

Shape - concatenated data:  (1259, 25) 

Sample of the filtered and combined dataframe


,Age,Gender,Country,state,self-employed,family history,treatment,work interfere,number of employees,remote work,...,anonymity,leave,mental health consequence,physical health consequence,coworkers,supervisor,mental health interview,physical health interview,mental vs physical,obs consequence
0,37,Female,United States,IL,NaN,No,Yes,Often,6-25,No,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,No,...,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
2,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,No,...,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,No,...,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,31,Male,United States,TX,NaN,No,No,Never,100-500,Yes,...,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No
5,33,Male,United States,TN,NaN,Yes,No,Sometimes,6-25,No,...,Don't know,Don't know,No,No,Yes,Yes,No,Maybe,Don't know,No
6,35,Female,United States,MI,NaN,Yes,Yes,Sometimes,1-5,Yes,...,No,Somewhat difficult,Maybe,Maybe,Some of them,No,No,No,Don't know,No
7,39,M,Canada,NaN,NaN,No,No,Never,1-5,Yes,...,Yes,Don't know,No,No,No,No,No,No,No,No
8,42,Female,United States,IL,NaN,Yes,Yes,Sometimes,100-500,No,...,No,Very difficult,Maybe,No,Yes,Yes,No,Maybe,No,No
9,23,Male,Canada,NaN,NaN,No,No,Never,26-100,No,...,Don't know,Don't know,No,No,Yes,Yes,Maybe,Maybe,Yes,No




Check the number of empty rows in the dataframe
Age                              0
physical health interview        0
mental health interview          0
supervisor                       0
coworkers                        0
physical health consequence      0
mental health consequence        0
leave                            0
anonymity                        0
seek help                        0
wellness program                 0
mental vs physical               0
care options                     0
tech company                     0
remote work                      0
number of employees              0
treatment                        0
family history                   0
Country                          0
Gender                           0
benefits                         0
obs consequence                  0
self-employed                   18
work interfere                 264
state                          515
dtype: int64 

Values in the tech company column:  
 tech company
Yes      

,Age,Gender,Country,state,self-employed,family history,treatment,work interfere,number of employees,remote work,...,anonymity,leave,mental health consequence,physical health consequence,coworkers,supervisor,mental health interview,physical health interview,mental vs physical,obs consequence
0,37,Female,United States,IL,NaN,No,Yes,Often,6-25,No,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,No,...,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
2,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,No,...,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,No,...,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,31,Male,United States,TX,NaN,No,No,Never,100-500,Yes,...,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No




Examine the Age values in the dataframe: 
 count          1259
mean       79428148
std     -2147483648
min           -1726
25%              27
50%              31
75%              36
max     -2147483648
Name: Age, dtype: int32 

Examine age values in the dataframe after age irregularities have been addressed: 
 count    1259
mean       32
std         7
min        18
25%        27
50%        31
75%        36
max        72
Name: Age, dtype: int32 

Contents of the Gender column after reducing gender categories:  
 Gender
Male      996
Female    247
Other      16
Name: count, dtype: int64 

Check the number of empty rows in the dataframe again:  
 Age                            0
physical health interview      0
mental health interview        0
supervisor                     0
coworkers                      0
physical health consequence    0
mental health consequence      0
leave                          0
anonymity                      0
seek help                      0
wellness progra